In [2]:
import joblib
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [ ]:
from bangla_pdf_ocr import process_pdf
path = "HSC26-Bangla1st-Paper.pdf"
output_file = "HSC26-Bangla1st-Paper.txt"
extracted_text = process_pdf(path, output_file)
print(extracted_text)

In [ ]:
file_path = "HSC26-Bangla1st-Paper.txt"

loader = TextLoader(file_path, encoding="utf-8")
documents = loader.load()

In [3]:
import pandas as pd
df = pd.read_excel("bangla_stopwords.xlsx", index_col=None)
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()
#df.to_dict(orient="dict")

,word_list
0,অই
1,অগত্যা
2,অত: পর
3,অতএব
4,অথচ


In [95]:
stopwords = set(df['word_list'].dropna().str.strip())
joblib.dump(stopwords, 'stopwords.pkl')
print(stopwords)

{'কোথা', 'কয়েক', 'রয়েছে', 'একে', 'পরেও', 'ওকে', 'দেখাচ্ছে', 'সাত', 'এছাড়াও', 'তাঁর', 'নাম', 'পড়া', 'চেয়েছিলেন', 'সেটা', 'কমনে', 'স্টপ', 'কেন', 'মস্ত', 'সংক্ষেপে', 'দিয়েছে', 'মানে', 'যেন', 'গত', 'উপরে', 'সত্যিই', 'আগামী', 'পূর্বে', 'বলেছেন', 'সামগ্রিক', 'মত', 'স্বয়ং', 'প্রায়ই', 'বর্তমান', 'যেখানে', 'প্রান্ত', 'স্থাপিত', 'হওয়ায়', 'তারপরেও', 'যাওয়ার', 'হয়নি', 'তিনিও', 'অসদৃশ', 'আছে', 'হলেই', 'হোক', 'পুরোপুরি', 'কিছুটা', 'এরকম', 'মতোই', 'ওর', 'রয়েছে', 'এখনো', 'শীঘ্র', 'রকম', 'করবে', 'তথা', 'দেখেন', 'যখনই', 'থেকেই', 'পরেই', 'বিভিন্ন', 'কোনও', 'জনের', 'এবার', 'মামলা', 'দেখতে', 'দিন', 'গিয়ে', 'নিজের', 'জানানো', 'বিশেষণ', 'হয়েছে', 'সেই', 'ভবিষ্যতে', 'উঠা', 'তোমার', 'সংখ্যা', 'আজ', 'যোগ', 'থাকে', 'দিতে', 'কেখা', 'বিশেষভাবে', 'নিজেকে', 'সম্প্রতি', 'এ', 'করায়', 'কিছুনা', 'ক্রম', 'বেশী', 'যেতে', 'সেরা', 'কেউনা', 'করবেন', 'করেছেন', 'তাই', 'এখনও', 'সংশ্লিষ্ট', 'ফলে', 'মুখ', 'করেছিলেন', 'দু', 'পরবর্তী', 'সর্বত্র', 'তুমি', 'করাত', 'করিয়ে', 'এদের', 'নয়', 'হিসাবে', 'পারা', 'ইশারা', 'যেমন', '

In [97]:
st = joblib.load('stopwords.pkl')
print(st)

{'কয়েক', 'কোথা', 'রয়েছে', 'একে', 'পরেও', 'ওকে', 'দেখাচ্ছে', 'সাত', 'এছাড়াও', 'তাঁর', 'নাম', 'পড়া', 'চেয়েছিলেন', 'সেটা', 'কমনে', 'স্টপ', 'কেন', 'মস্ত', 'সংক্ষেপে', 'দিয়েছে', 'মানে', 'যেন', 'গত', 'উপরে', 'সত্যিই', 'আগামী', 'পূর্বে', 'বলেছেন', 'সামগ্রিক', 'মত', 'স্বয়ং', 'প্রায়ই', 'বর্তমান', 'যেখানে', 'প্রান্ত', 'স্থাপিত', 'হওয়ায়', 'যাওয়ার', 'হয়নি', 'তারপরেও', 'তিনিও', 'অসদৃশ', 'আছে', 'হলেই', 'হোক', 'পুরোপুরি', 'কিছুটা', 'এরকম', 'মতোই', 'ওর', 'রয়েছে', 'এখনো', 'শীঘ্র', 'রকম', 'করবে', 'তথা', 'দেখেন', 'যখনই', 'থেকেই', 'পরেই', 'বিভিন্ন', 'কোনও', 'জনের', 'এবার', 'মামলা', 'দেখতে', 'দিন', 'গিয়ে', 'নিজের', 'জানানো', 'বিশেষণ', 'হয়েছে', 'সেই', 'ভবিষ্যতে', 'উঠা', 'তোমার', 'সংখ্যা', 'আজ', 'যোগ', 'থাকে', 'দিতে', 'কেখা', 'নিজেকে', 'সম্প্রতি', 'এ', 'করায়', 'কিছুনা', 'ক্রম', 'বেশী', 'যেতে', 'সেরা', 'কেউনা', 'করবেন', 'করেছেন', 'তাই', 'এখনও', 'সংশ্লিষ্ট', 'ফলে', 'মুখ', 'করেছিলেন', 'দু', 'পরবর্তী', 'সর্বত্র', 'তুমি', 'করাত', 'করিয়ে', 'এদের', 'নয়', 'হিসাবে', 'পারা', 'ইশারা', 'যেমন', 'একদা', 'প্রদত

In [ ]:
# Join all pages' contents into one string
all_text = " ".join([doc.page_content for doc in documents])
joblib.dump(all_text, 'all_text.pkl')

In [ ]:
print(all_text[:500])

In [9]:
import re
import unicodedata
     
def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in stopwords]
    return " ".join(filtered)

def bangla_sentence_split(text):
    return re.split(r"(?<=[।!?])\s+", text.strip())

def clean_bangla_text(text):

    text = unicodedata.normalize("NFC", text)

    # Remove English digits and symbols
    text = re.sub(r"[a-zA-Z0-9]", "", text)

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)

    # Remove extra punctuation/special chars except Bangla ones
    text = re.sub(r"[“”\"\'\(\)\[\]\{\}<>\|;:…~`_+=@#$%^&*\\]", "", text)

    # Normalize spacing
    text = re.sub(r"\s+", " ", text)

    return text.strip()

def bangla_chunker(texts, chunk_size=600, chunk_overlap=70): 
    sentences = bangla_sentence_split("".join(texts))
    chunks = []
    current_chunk = []
    total_len = 0

    for sent in sentences:
        sent_len = len(sent)
        if total_len + sent_len <= chunk_size:
            current_chunk.append(sent)
            total_len += sent_len
        else:
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)

            # Get last `chunk_overlap` characters for overlap
            overlap_text = chunk_text[-chunk_overlap:]

            # Start new chunk with overlap + current sentence
            current_chunk = [overlap_text + " " + sent]
            total_len = len(current_chunk[0])

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [75]:
def preprocess_and_chunk(text, chunk_size=500, chunk_overlap=70):
    # Step 1: Clean
    cleaned = clean_bangla_text(text)
    
    # Step 2: Remove stopwords
    filtered = remove_stopwords(cleaned)
    
    # Step 3: Chunk
    chunks = bangla_chunker([filtered], chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    return chunks

# Use it
chunks = preprocess_and_chunk(all_text)

In [76]:
print(chunks[1])  # Display the first 3 chunks

 বাবা জীবিকা নির্বাহ করতেন? ক ডাক্তারি খ ওকালতি গ মাস্টারি ঘ ব্যবসা ২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার- ক প্রতিপত্তি খ প্রভাব গ বিচক্ষণতা ঘ কুট বুদ্ধি নিচের অনুচ্ছেদটি পড়ে ৩ ৪ সংখ্যক প্রশ্নের দাও। পিতৃহীন দীপুর চাচাই পরিবারের কর্তা। দীপু শিক্ষিত সিদ্ধান্ত ক্ষমতা না। চাচা বিয়ের উদ্যোগ নিলেও যৌতুক বাড়াবাড়ি কারণে কন্যার পিতা অপমানিত বোধ বিয়ের আলোচনা ভেঙে দেন। দীপু মেয়েটির ছবি মুগ্ধ চাচাকে পারেননি। ৩। দীপুর চাচার অপরিচিতা গল্পের চরিত্রের মিল আছে?


In [77]:
for i, chunk in enumerate(chunks[:5]):
    print(f"Chunk {i}:\n{chunk}\n")

Chunk 0:
অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়, ৮ নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী ওঠার সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে। ৮ তৎকালীন সমাজ-সভ্যতা মানবতার অবমাননা সম্পর্কে পারবে। ৮ তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে পারবে। ৮ তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্ট্য সম্পর্কে জ্ঞানলাভ করবে। পারবে। ৮ আশা বেঁচে থাকে- অনুপমের দৃষ্টান্তে মানবজীবনের চিরন্তন সত্যদর্শন সম্পর্কে জ্ঞানলাভ করবে। ছু প্রাক-মূল্যায়ন ১। অনুপমের বাবা জীবিকা নির্বাহ করতেন? ক ডাক্তারি খ ওকালতি গ মাস্টারি ঘ ব্যবসা ২।

Chunk 1:
 বাবা জীবিকা নির্বাহ করতেন? ক ডাক্তারি খ ওকালতি গ মাস্টারি ঘ ব্যবসা ২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার- ক প্রতিপত্তি খ প্রভাব গ বিচক্ষণতা ঘ কুট বুদ্ধি নিচের অনুচ্ছেদটি পড়ে ৩ ৪ সংখ্যক প্রশ্নের দাও। পিতৃহীন দীপুর চাচাই পরিবারের কর্তা। দীপু শিক্ষিত সিদ্ধান্ত ক্ষমতা না। চাচা বিয়ের উদ্যোগ নিলেও যৌতুক বাড়াবাড়ি কারণে কন্যার পিতা অপমানিত বোধ বিয়ের আলোচনা ভেঙে দেন। দীপু মেয়েটির ছবি মুগ্ধ চাচাকে পারেননি। ৩। দীপুর চাচার অপরিচিতা গল্পের চরিত্রের মিল আছে?

Chunk 2:
মুগ্ধ চ

In [ ]:
#embeddings = HuggingFaceEmbeddings(model_name="mahedi420/Bangla-bert-improved-version")

#embeddings = HuggingFaceEmbeddings(model_name="shihab17/bangla-sentence-transformer")

# embeddings = HuggingFaceEmbeddings(model_name="banglagov/banBERT-Base")

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

vector_store = Chroma(
    collection_name="bangla-pdf-base",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

No sentence-transformers model found with name banglagov/banBERT-Base. Creating a new one with mean pooling.


In [79]:
from langchain.schema import Document

# Convert chunks to Document objects
docs = [Document(page_content=chunk) for chunk in chunks]

# Add to vector store
vector_store.add_documents(docs)

['fde7a34b-a8e1-486e-8960-2531c102f96f',
 '1efc7f2f-15e0-4d86-b7c0-b86c1c0f376f',
 'ea381969-1cc5-4b23-988a-75b2e9771f1c',
 'a735dba9-c10a-4c38-b4eb-d4c3be11904d',
 '4ecab160-0c73-4ae7-8bf5-4ee8a240f5bb',
 '8d617025-98ae-4bfc-8d61-f8cdbe5c3cc9',
 '8b3531c2-8260-4eb4-9d68-b79e9b5c7b94',
 '72185c46-0b3a-463f-a478-c4df37548ca3',
 'c027585a-ddc2-45d1-8568-f22e4098e4cd',
 '18450258-46b4-4a96-aba3-2f52479262a3',
 '76a7c7ab-2155-48f5-8981-cbf0c77a2ba9',
 '3c559f53-edc0-440d-bb11-cbba998772f1',
 'dc7eb57d-516e-4fe2-bcfc-d51f82f43ef0',
 'a9dd9d67-961e-43f9-9b65-32b52fe17340',
 'd5b366d0-5fa1-4acc-b721-ce1f66b31653',
 'b50f215b-a63e-41ae-aa8d-08a129391fdd',
 'aef1babd-1fde-4250-90c9-eb4217ec2c94',
 'eb0d47c3-0865-4191-a1de-ff182eb97818',
 'dec1eb4a-3562-4769-8687-7f4d3ececdf3',
 '10b31e92-b122-4315-9c4a-c15b661d1b63',
 'c9043cb2-f941-433c-9416-1e7e7711fded',
 'a62797ea-65e0-4091-9161-9d814d2b74e4',
 'd7e8cf6c-a401-4fea-8698-a7e0d486fd14',
 'f0d17bda-5ce5-4ba2-b7d7-2fbb605531af',
 'c7aa973e-a3a5-

gorq = "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"

In [ ]:
GROQ_API_KEY= "past_api_key_here"
chat = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0,
    max_retries=3,
    api_key=GROQ_API_KEY,
)

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")
retriever = Chroma(
    collection_name="bangla-pdf-base",
    embedding_function=embeddings,
    persist_directory="./data/processed/chroma_langchain_db"
).as_retriever()

C:\Users\Imtiaz\AppData\Local\Temp\ipykernel_12752\904075265.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")


In [4]:
# custom_prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template=(
#         "নিচের তথ্যটি মনোযোগ দিয়ে পড়ুন এবং প্রশ্নের উত্তর দিন। "
#         "তথ্য ছাড়া অনুমান করবেন না। শুধুমাত্র সঠিক উত্তর দিন।\n\n"
#         "তথ্য:\n{context}\n\n"
#         "প্রশ্ন: {question}\n"
#         "উত্তর:"
#     )
# )

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Read the following context carefully and answer the question. "
        "The question may be in Bangla, English. "
        "Answer strictly based on the context without making assumptions. Just provide answer, don't assume anyhting.\n\n"
        "Respond in the same language as the question.\n\n"

        # Few-shot Example 1: English
        "Example 1 (English):\n"
        "Context:\nBangladesh became an independent country in 1971 after a war of liberation.\n"
        "Question: When did Bangladesh gain independence?\n"
        "Answer: Bangladesh gained independence in 1971.\n\n"

        # Few-shot Example 2: Bangla
        "Example 2 (Bangla):\n"
        "Context:\nকাজী নজরুল ইসলাম বাংলাদেশের জাতীয় কবি। তিনি বিদ্রোহী কবি নামেও পরিচিত।\n"
        "প্রশ্ন: বাংলাদেশের জাতীয় কবি কে?\n"
        "উত্তর: বাংলাদেশের জাতীয় কবি কাজী নজরুল ইসলাম।\n\n"

        # Actual Input
        "Now answer the following:\n"
        "Context:\n{context}\n\n"
        "Question: {question}\n"
        "Answer:"
    )
)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_prompt}
) 
res = qa.invoke("কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে")
#res=qa.invoke("বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?")
#res=qa.invoke("অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?")
answer = res['result'].strip()
print(answer)